In [1]:
import numpy as np
import pandas as pd
from hyperopt import hp, fmin, tpe
from functions import CreatDataGBM
import xgboost as xgb

In [2]:
X_train,Y_train,X_val,Y_val,Weight = CreatDataGBM(False)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
def NWRMSLE_exp(preds, dtrain):
    labels = dtrain.get_label()
    preds = np.exp(preds)
    temp1 = preds/(preds+1)
    temp2 = np.log(preds+1) - labels
    return temp2*temp1, temp1*temp1 + temp2*temp1*(1-temp1)

In [4]:
def NWRMSLE_relu(preds, dtrain):
    labels = dtrain.get_label()
    preds2 = np.maximum(preds,0)
    inc = preds>0
    temp1 = inc/(1+preds2)
    temp2 = np.log(preds2+1) - labels
    sq = temp1*temp1
    return temp2*temp1, sq - temp2*sq

In [5]:
objective_fun = [(NWRMSLE_exp,np.exp),(NWRMSLE_relu,lambda x:np.maximum(x,0))]

In [9]:
param = {}
param['eta'] = 0.05
param['max_depth'] = 10
param['min_child_weight'] = 1
param['tree_method'] = 'hist'
param['subsample'] = 0.133448450345
param['colsample_bytree'] = 0.367127373623
param['gamma'] = 5.18910389561

In [10]:
best_numberStep_list = [1000,
 900,
 500,
 200,
 200,
 500,
 1000,
 1000,
 1000,
 300,
 600,
 1000,
 200,
 900,
 700,
 1000]
Y_test_hat = np.zeros_like(Y_val)
obj,link = objective_fun[1]

In [11]:
for i in range(16):
    dtrain = xgb.DMatrix(
        X_train, label=Y_train[:, i],
        weight=np.tile(Weight,16)
    )
    dval = xgb.DMatrix(
        X_val,weight=Weight)
    model = xgb.train(param, dtrain, best_numberStep_list[i],obj=obj)
    Y_test_hat[:,i] = link(model.predict(dval,output_margin=True))

In [14]:
pd.DataFrame({'id':Y_val.flatten(),'unit_sales':Y_test_hat.flatten()})\
  .to_csv('FinalGBM.csv',index=False)

In [12]:
old = pd.read_csv('SS_GBM.csv',dtype={'id': 'int32','unit_sales': 'float32'})

In [46]:
SI = pd.DataFrame({'id':Y_val.flatten(),'unit_sales':Y_test_hat.flatten()})  

In [20]:
SI1 = pd.merge(old,SI,'left','id')

In [24]:
SI1['unit_sales'] = np.where(np.isnan(SI1.unit_sales_y),SI1.unit_sales_x,SI1.unit_sales_y)

In [28]:
SI1.drop(['unit_sales_x','unit_sales_y'],1).to_csv('FinalGBM_all.csv',index=False)